In [4]:
import requests
import openai
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os
from bs4 import BeautifulSoup
from openpyxl import load_workbook
from datetime import datetime, timedelta

os.chdir("D:/workshops/temp_World Bank Jobs Application Dashboard and Automate")

## Step 1. Loop through the table pages to extract all rows

In [5]:
# Open and retrieve the initial page's table
driver = webdriver.Chrome()
driver.get('https://worldbankgroup.csod.com/ats/careersite/search.aspx?site=1&c=worldbankgroup&sid=%5e%5e%5eFLGscZMYY2RrwVaMR%2ftHYw%3d%3d')
time.sleep(3)
print("Page Number 1 Displayed")
page_s = driver.page_source
soup = BeautifulSoup(page_s, 'html.parser')

list_all = []

# Home Page
for row in soup.find('table', attrs={'class':'results-table'}).find('tbody').find_all('tr'):
    list_in = []
    list_in.append(row.find('a').text)
    list_in.append('https://worldbankgroup.csod.com/ats/careersite/' + row.find('a')['href'])
    i = 1 
    list_in.append(row.find_all('span')[i].text)
    i += 1
    list_in.append(row.find_all('span')[i].text)
    i += 1
    if 'ctl05_displayField' in row.find_all('span')[i]['id']:
        list_in.append(row.find_all('span')[i].text)
        i += 1
    else:
        list_in.append(' ')
    list_in.append(row.find_all('span')[i].text)
    i += 1
    list_in.append(row.find_all('span')[i].text)
    i += 1
    list_in.append(row.find_all('span')[i].text)
    list_all.append(list_in)

# Other Pages
page_count = 2
try:
    for i in range(0,20):
        driver.find_element(By.CSS_SELECTOR, 'a[data-tag="nextPageLink"]').click()
        time.sleep(3)
        page_s = driver.page_source
        soup = BeautifulSoup(page_s, 'html.parser')

        for row in soup.find('table', attrs={'class':'results-table'}).find('tbody').find_all('tr'):
            list_in = []
            list_in.append(row.find('a').text)
            list_in.append('https://worldbankgroup.csod.com/ats/careersite/' + row.find('a')['href'])
            i = 1 
            list_in.append(row.find_all('span')[i].text)
            i += 1
            list_in.append(row.find_all('span')[i].text)
            i += 1
            if 'ctl05_displayField' in row.find_all('span')[i]['id']:
                list_in.append(row.find_all('span')[i].text)
                i += 1
            else:
                list_in.append(' ')
            list_in.append(row.find_all('span')[i].text)
            i += 1
            list_in.append(row.find_all('span')[i].text)
            i += 1
            list_in.append(row.find_all('span')[i].text)
            list_all.append(list_in)
        print("Page Number " + str(page_count) + " Displayed")
        page_count += 1
except:
    print("No More Pages!")

Page Number 1 Displayed
Page Number 2 Displayed
Page Number 3 Displayed
Page Number 4 Displayed
Page Number 5 Displayed
Page Number 6 Displayed
Page Number 7 Displayed
Page Number 8 Displayed
Page Number 9 Displayed
Page Number 10 Displayed
No More Pages!


## Step 2. Read current status and combine, then export

In [6]:
df = pd.DataFrame(list_all)
df.columns = ['title','detail','instit','code','grade','location','topic','ddl']
df['ddl_date'] = pd.to_datetime(df['ddl'],format='%m/%d/%Y')
df['status'] = ''
status_update = pd.read_excel('wb dashboard.xlsm')[['status_update','code']]
merged_df = pd.merge(df,status_update, how='left', on='code')

merged_df = merged_df[(merged_df['location']=='Washington, DC,United States') & (merged_df['ddl_date']>datetime.today() - timedelta(days=1))]
merged_df = merged_df[~merged_df['title'].str.contains('Internship')]
merged_df['status'] = merged_df['status_update'] 
merged_df = merged_df.sort_values(by=['ddl_date','status']).drop(['ddl_date','status_update'], axis=1)
merged_df.to_csv('data.csv', index=False)

In [7]:
# dfs = pd.read_html("https://worldbankgroup.csod.com/ats/careersite/search.aspx?site=1&c=worldbankgroup&sid=%5e%5e%5eFLGscZMYY2RrwVaMR%2ftHYw%3d%3d")
# dfs[3]